In [132]:
import numpy as np
data = np.load('C:\\Users\\zihan\\Desktop\\Files\\Research\\bryankelly_without_missing.npz')
processed_data = data['processed_data']
print(processed_data.shape)
T, N, M = processed_data.shape
returns = processed_data[:, :, 1]
# The dimension of returns: T*N
chara_data = processed_data[:, :, 2:]
for i in range(T):
    for j in range(chara_data.shape[2]):
        chara_data[i, :, j] = np.argsort(chara_data[i, :, j])
        chara_data[i, :, j] = (chara_data[i, :, j] + 1) / (N + 1)
        chara_data[i,:,j]= (chara_data[i,:,j]- np.mean(chara_data[i,:,j]))/np.sum(np.abs(chara_data[i,:,j]- np.mean(chara_data[i,:,j])))

# The dimension of characteristics: 

(360, 304, 42)


In [133]:
# Get the interaction characteristics
def getinteraction(vector):
    output= np.zeros((int(len(vector)*(len(vector)+1)/2+2*len(vector))))
    whichmatter= []
    count= 0
    for i in range(len(vector)):
        for j in range(i, len(vector)):
            output[count]= vector[i]*vector[j]
            count+=1 
            whichmatter+= [(i,j)]
    for i in range(len(vector)):
        output[count]= vector[i]
        whichmatter+= [(i)]
        count+= 1
        output[count]= vector[i]**3
        whichmatter+= [(i,i,i)]
        count+= 1
    return output, whichmatter


&0.06&0.06&0.09&0.09&0.09&0.10&0.10&0.10&0.13&0.13


In [134]:
def getMatrix(inputmatrix):
    output= np.zeros((inputmatrix.shape[0],int(inputmatrix.shape[1]*(inputmatrix.shape[1]+1)/2+2*inputmatrix.shape[1])))
    bound= int(inputmatrix.shape[1]*(inputmatrix.shape[1]+1)/2)
    for i in range(inputmatrix.shape[0]):
        output[i,:],_= getinteraction(inputmatrix[i,:])
    # output[:,:bound]= output[:,:bound]- np.mean(output[:,:bound], axis= 0, keepdims= True)
    # output[:,:bound]= output[:,:bound]/np.sum(np.abs(output[:,:bound]), axis= 0, keepdims= True)
    # output= output- np.mean(output, axis= 0, keepdims= True)
    # output= output/np.sum(np.abs(output), axis= 0, keepdims= True)
    return output

&0.06&0.06&0.09&0.09&0.09&0.10&0.10&0.10&0.13&0.13


In [135]:
portfolio= np.zeros((T-1, int(chara_data.shape[2]*(chara_data.shape[2]+1)/2+2*chara_data.shape[2])))
for i in range(T-1):
    portfolio[i,:]= np.dot(returns[i+1,:], getMatrix(chara_data[i,:]))
# portfolio= np.zeros((T-1, chara_data.shape[2]))
# for i in range(T-1):
#     portfolio[i,:]= np.dot(returns[i+1,:], (chara_data[i,:]))

&0.06&0.06&0.09&0.09&0.09&0.10&0.10&0.10&0.13&0.13


In [136]:
portfolio_demean = portfolio - np.mean(portfolio, axis=0)
# Dimension of portfolio: T*M
covmat = portfolio_demean.T.dot(portfolio_demean) / (T - 2)
variance, loading = np.linalg.eig(covmat)
output= []
loading_chosen = loading[:, :11]
Factor = portfolio.dot(loading_chosen)
for i in range(1, 11):
    mu = np.mean(Factor[:,:i], axis=0)
    covmat_factor = np.cov(Factor[:,:i].T)
    if i==1: 
        output.append (abs(mu)*covmat_factor**(-1/2))
    else:
        output.append(np.sqrt(np.dot(np.dot(mu, np.linalg.inv(covmat_factor)), mu.T)))
twodecimals = ["&%.2f" % np.real(v) for v in output]
print (''.join(twodecimals))


&0.06&0.06&0.09&0.09&0.09&0.10&0.10&0.10&0.13&0.13


In [33]:
print (portfolio.shape)

(360, 900)


In [42]:
output,_= getinteraction(chara_data[0,0,:])
print (np.count_nonzero(output))

900


In [131]:
A=1.0*np.reshape(np.arange(6),(3,2))
output= getMatrix(A)
for i in range(3):
    A[i,:]= (A[i,:]- np.mean(A[i,:]))/np.sum(np.abs(A[i,:]- np.mean(A[i,:])))

In [130]:
print (output)
print (A)

[[   0.    0.    1.    0.    0.    1.    1.]
 [   4.    6.    9.    2.    8.    3.   27.]
 [  16.   20.   25.    4.   64.    5.  125.]]
[[-0.5  0.5]
 [-0.5  0.5]
 [-0.5  0.5]]


In [68]:
output, _= getinteraction(A[1,:])

[ -6.55730609e-03   2.35889889e-03   2.31561634e-03   2.27233380e-03
   2.22905125e-03   2.18576870e-03   2.14248615e-03   2.09920360e-03
   2.05592105e-03   2.01263850e-03   1.96935596e-03   1.92607341e-03
   1.88279086e-03   1.83950831e-03   1.79622576e-03   1.75294321e-03
   2.40218144e-03   1.70966066e-03   2.44546399e-03   2.53202909e-03
   3.18126731e-03   3.13798476e-03   3.09470222e-03   3.05141967e-03
   3.00813712e-03   2.96485457e-03   2.92157202e-03   2.87828947e-03
   2.83500693e-03   2.79172438e-03   2.74844183e-03   2.70515928e-03
   2.66187673e-03   2.61859418e-03   2.57531163e-03   2.48874654e-03
   3.22454986e-03   1.66637812e-03   1.57981302e-03   7.14162050e-04
   6.70879501e-04   6.27596953e-04   5.84314404e-04   5.41031856e-04
   4.97749307e-04   4.54466759e-04   4.11184211e-04   3.67901662e-04
   3.24619114e-04   2.81336565e-04   2.38054017e-04   1.94771468e-04
   1.51488920e-04   1.08206371e-04   7.57444598e-04   1.62309557e-03
   8.00727147e-04   8.87292244e-04